In [1]:
# Packages
from bs4 import BeautifulSoup as soup
import pandas as pd
import time as t
import random
import math

In [2]:
base_url = 'https://www.facebook.com/marketplace/108145369212869/search?'
# Parameters
estimated_desired_results = 168
min_price = 1000
make = 'suzuki'
model = 'sv650'
# Full url
full_url = f"{base_url}minPrice={min_price}&query={make}%20{model}&exact=false"

In [3]:
# Set up selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
options = Options()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                          options=options)

# Visit web site
driver.get(full_url)

In [4]:
# Scroll
scroll_count = int(math.ceil(estimated_desired_results/28))
for _ in range(scroll_count):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    t.sleep(1+math.floor(random.random()*3))

In [5]:
# Getting the HTML
html = driver.page_source
html_soup = soup(html, 'html.parser')
# Closing the browser
driver.quit()

In [6]:
# Getting the information v2

url_class = 'x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1lku1pv'
url_divs = html_soup('a', class_= url_class)

titles = []
prices = []
milages = []
urls = []

for url in url_divs:
    title_element = url.find('span', class_= 'x1lliihq x6ikm8r x10wlt62 x1n2onr6')
    price_element = url.find('span', class_='x193iq5w xeuugli x13faqbe x1vvkbs x10flsy6 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x1tu3fi x3x7a5m x1lkfr7t x1lbecb7 x1s688f xzsf02u')
    milage_elements = url.find_all('span', class_='x1lliihq x6ikm8r x10wlt62 x1n2onr6 xlyipyv xuxw1ft')
    title = title_element.text.strip() if title_element else 'none'
    titles.append(title)
    price = price_element.text.strip() if price_element else 'none'
    prices.append(price)
    if milage_elements:
        for milage_element in milage_elements:
            milage = milage_element.text.strip()
            milages.append(milage)
    else: milages.append('none')
    url_element = url.get('href')
    urls.append(url_element)

In [7]:
# Clean milages
milages_2 = []    
if milages[0][0].isdecimal():
     milages_2.insert(0,'None')    
for new_elem in milages:
    milages_2.append(new_elem)
    if new_elem == '':
        milages_2.pop()
    length = len(milages_2)    
    if length%2 == 0 and  new_elem[-3:] != ' km' and new_elem[-6:] != ' miles':
        milages_2.insert(-1,'0k km')
    
if milages_2[-1][-3:] != ' km' and milages_2[-1][-6:] != ' miles':
        milages_2.append('0k km')

In [8]:
# Miles to km

for i in range(len(milages_2)):
    elem = milages_2[i]
    if elem[-6:] == ' miles':
        miles_to_km = float(elem[0:-7])*1.609
        temp = miles_to_km*100 //10
        if int((miles_to_km*100)//1) % 10 > 4:
            temp += 1  
        km = temp/10
        milages_2[i] = str(km)+'K km'

In [9]:
# milage_only(Kkms to kms numbre only) & city_only
milage_clean = []
city_clean = []

for i in range(len(milages_2)):
    elem = milages_2[i]
    if i % 2 == 0: city_clean.append(elem)
    else: milage_clean.append(int(float(elem[:-4])*1000))

In [10]:
# Price clean v2
prices_clean = []

for price in prices:
    new_price = ''
    for character in price:
        if character.isdecimal():
            new_price += (character)
    if len(new_price) < 6:
        prices_clean.append(new_price)
    else: prices_clean.append('0')

In [11]:
# Tests
def tests():
    assert len(milage_clean) == len(city_clean)
    assert len(milage_clean) == len(prices_clean)
    assert len(titles) == len(prices_clean)
    assert len(urls) == len(prices_clean)

tests()

In [12]:
# Data Frame
vehicle_list = []
make_list = [
    'acura', 'alfa', 'romeo', 'aston', 'martin', 'audi', 'bentley', 'bmw', 'buick',
    'cadillac', 'chevrolet', 'chrysler', 'dodge', 'ferrari', 'fiat', 'ford',
    'gmc', 'honda', 'hyundai', 'infiniti', 'jaguar', 'jeep', 'kia', 'lamborghini',
    'land', 'rover', 'lexus', 'lincoln', 'maserati', 'mazda', 'mclaren', 'mercedes', 'benz',
    'mini', 'mitsubishi', 'nissan', 'porsche', 'ram', 'rolls-royce', 'subaru', 'tesla',
    'toyota', 'volkswagen', 'volvo',
    'harley', 'davidson', 'ducati', 'yamaha', 'suzuki', 'kawasaki',
     'triumph', 'aprilia', 'mv agusta', 'kTM', 'indian',
    'vespa', 'norton', 'royal', 'enfield', 'zero'
]

for i in range(len(prices_clean)):
    vehicle_dict = {}
    vehicle_dict['Year'] = 'none'
    vehicle_dict['Make'] = 'none'
    vehicle_dict['Model'] = 'none'
    vehicle_dict['Price'] = int(prices_clean[i])
    vehicle_dict['Mileage'] = int(milage_clean[i])
    vehicle_dict['Url'] = 'https://www.facebook.com/'+urls[i]
    elem_model = ''
    elem_model_2 = ''
    elem_make = ''
    title_split = titles[i].split()

    j = 0
    for k in range(len(title_split)):
        elem = title_split[k]

        if elem.isdecimal() and int(elem) > 1950:
            if int(elem) < 2025:
                vehicle_dict['Year'] = title_split[k]
            else: vehicle_dict['Mileage'] = title_split[k]
        elif elem.isalpha() and elem.lower() in make_list:
            elem_make += str(elem)
        elif elem.lower() == 'km':
            pass   
        elif j > 0: 
            elem_model += ' '+str(elem)
        else: 
            elem_model += str(elem)
            j += 1

# No uppercase
    
    if elem_model == '':
        elem_model = 'none'
    if elem_make == '':
        elem_make = 'none'

    elem_make = elem_make.lower()

    for character in elem_model:
        if character.isalpha():
            elem_model_2 += character.lower()
        else: elem_model_2 += character    
        
    vehicle_dict['Model'] = elem_model_2
    vehicle_dict['Make'] = elem_make
    vehicle_list.append(vehicle_dict)

vehicle_df = pd.DataFrame(vehicle_list)

In [13]:
# Cleaning of DF
# Conditions
right_make = vehicle_df['Make'].str.lower() == make
right_model = (vehicle_df['Model'].str.contains(model)) 
desired_price = vehicle_df['Price'].astype('Int64') > min_price                                                             
filtered_df = vehicle_df[right_make & right_model & desired_price]
print('Number of entries : '+str(len(urls)))
print('Number of filtered entries : '+str(len(filtered_df)))
filtered_df

Number of entries : 168
Number of filtered entries : 55


,Year,Make,Model,Price,Mileage,Url
0,2005,suzuki,sv650,2500,43000,https://www.facebook.com//marketplace/item/733...
1,2001,suzuki,sv650,2000,67000,https://www.facebook.com//marketplace/item/163...
2,2015,suzuki,sv650s (abs),4800,34000,https://www.facebook.com//marketplace/item/336...
4,2011,suzuki,sv650,5000,17000,https://www.facebook.com//marketplace/item/133...
6,2011,suzuki,sv650s,3499,37000,https://www.facebook.com//marketplace/item/101...
7,2013,suzuki,sv650s,4750,34000,https://www.facebook.com//marketplace/item/882...
8,2004,suzuki,sv650s,2500,60000,https://www.facebook.com//marketplace/item/148...
10,2000,suzuki,sv650s,2200,40000,https://www.facebook.com//marketplace/item/103...
12,2002,suzuki,sv650,1800,62000,https://www.facebook.com//marketplace/item/645...
13,2003,suzuki,sv650,3000,56000,https://www.facebook.com//marketplace/item/840...


In [14]:
# Writing the csv file
#csv_file_path = make+'_'+model+'_marketplace_data.csv'
#filtered_df.to_csv(csv_file_path)